In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [965 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,064 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,233 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,085 kB]
Hit:13 https://ppa.launch

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

# Set the partitions to 4 or 8.
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [ ]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url_listings = "http://data.insideairbnb.com/united-states/ny/new-york-city/2023-09-05/data/listings.csv.gz"
spark.sparkContext.addFile(url_listings)
listings_df = spark.read.csv(SparkFiles.get("listings.csv.gz"), sep=",", header=True)

# Create a lookup table for calendar.
url_calendar="http://data.insideairbnb.com/united-states/ny/new-york-city/2023-09-05/data/calendar.csv.gz"
spark.sparkContext.addFile(url_calendar)
calendar_df = spark.read.csv(SparkFiles.get("calendar.csv.gz"), sep=",", header=True)

# Create a lookup table for the airport codes.
url_reviews ="http://data.insideairbnb.com/united-states/ny/new-york-city/2023-09-05/data/reviews.csv.gz"
spark.sparkContext.addFile(url_reviews)
reviews_df = spark.read.csv(SparkFiles.get("reviews.csv.gz"), sep=",", header=True)


In [ ]:
# Look over the listings data.
listings_df.show()

+------------------+--------------------+--------------+------------+-----------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+--------------------+-------------------------+------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+------------------+--------------------+--------------------+--------------------+-------------+---------+------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------

In [ ]:
# Look over the data for calendar.
calendar_df.show()

+----------+----------+---------+-------+--------------+--------------+--------------+
|listing_id|      date|available|  price|adjusted_price|minimum_nights|maximum_nights|
+----------+----------+---------+-------+--------------+--------------+--------------+
|      2595|2023-09-05|        t|$240.00|       $240.00|            30|          1125|
|      2595|2023-09-06|        t|$240.00|       $240.00|            30|          1125|
|      2595|2023-09-07|        t|$240.00|       $240.00|            30|          1125|
|      2595|2023-09-08|        t|$240.00|       $240.00|            30|          1125|
|      2595|2023-09-09|        t|$240.00|       $240.00|            30|          1125|
|      2595|2023-09-10|        t|$240.00|       $240.00|            30|          1125|
|      2595|2023-09-11|        t|$240.00|       $240.00|            30|          1125|
|      2595|2023-09-12|        t|$240.00|       $240.00|            30|          1125|
|      2595|2023-09-13|        t|$240.00|  

In [ ]:
# Look over the review data.
reviews_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          listing_id|                  id|                date|         reviewer_id|       reviewer_name|            comments|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                2595|               17857|          2009-11-21|               50679|                Jean|Notre séjour de t...|
|<br/>Nous avons a...| propre et bien s...|                null|                null|                null|                null|
|<br/>Petits souci...| et il manquait l...| le liquide vaiss...| nous les avons r...| donc une premièr...|                null|
|<br/>Si j'ai la p...| je reprendrai à ...|                null|                null|                null|                null|
|           <br/>Jean|                null|                null|                null|                nul

In [ ]:
import pandas as pd

In [ ]:
listings_df.count()

64949

In [ ]:
calendar_df.count()

14399996

In [ ]:
reviews_df.count()

1107152

In [ ]:
listing_columns = ['id','listing_url','name','host_id','host_url','host_name','host_since','host_is_superhost','host_listings_count','host_total_listings_count','neighbourhood_cleansed','neighbourhood_group_cleansed','latitude','longitude','room_type','accommodates','bathrooms_text','bedrooms','beds','amenities','price','minimum_nights','maximum_nights','has_availability','number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d','first_review','last_review','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','calculated_host_listings_count','reviews_per_month']
calendars_columns = ['listing_id','date','available','price','adjusted_price']
reviews_columns = ['listing_id','id','date','reviewer_id','reviewer_name','comments']

In [ ]:
column_listings_df = listings_df[listing_columns]
column_calendars_df = calendar_df[calendars_columns]
column_reviews_df = reviews_df[reviews_columns]

In [ ]:
column_listings_df = column_listings_df.withColumnRenamed('id', 'listing_id')

In [ ]:
column_listings_df.show()

+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------------------------+----------------------+----------------------------+------------------+--------------------+--------------------+-------------+------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+----------------------+--------------------+--------------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+--------------------+------------------------------+--------------------+
|        listing_id|         listing_url|                name|             host_id|            host_url|           host_name|          host_since| host_is_superhost| host_lis

In [ ]:
from pyspark.sql import SparkSession


In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Assuming you have a PySpark DataFrame named 'column_listings_df'
# Drop rows with null values in the 'listing_id' column
column_listings_df = column_listings_df.dropna(subset=['listing_id'])

# Show the DataFrame after dropping null values
column_listings_df.show()

+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-------------------------+----------------------+----------------------------+------------------+--------------------+--------------------+-------------+------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+----------------------+--------------------+--------------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+--------------------+------------------------------+--------------------+
|        listing_id|         listing_url|                name|             host_id|            host_url|           host_name|          host_since| host_is_superhost| host_lis

In [ ]:
column_reviews_df = column_reviews_df.dropna(subset=['comments'])
column_reviews_df.show()

+----------+--------+----------+-----------+-------------+--------------------+
|listing_id|      id|      date|reviewer_id|reviewer_name|            comments|
+----------+--------+----------+-----------+-------------+--------------------+
|      2595|   17857|2009-11-21|      50679|         Jean|Notre séjour de t...|
|      2595|   19176|2009-12-05|      53267|         Cate|   Great experience.|
|      2595|   19760|2009-12-10|      38960|        Anita|I've stayed with ...|
|      2595|   34320|2010-04-09|      71130|      Kai-Uwe|We've been stayin...|
|      2595|   46312|2010-05-25|     117113|       Alicia|We had a wonderfu...|
|      2595| 1238204|2012-05-07|    1783688|       Sergey|     Hi to everyone!|
|      2595| 1293632|2012-05-17|    1870771|         Loïc|Jennifer was very...|
|      2595| 2022498|2012-08-18|    2124102|      Melanie|This apartment is...|
|      2595| 4682989|2013-05-20|     496053|         Eric|Jennifer's place ...|
|      2595|13193832|2014-05-21|   13685

In [ ]:
# Initialize a Spark session
spark = SparkSession.builder.appName("DataFrameJoin").getOrCreate()

# Assuming you have DataFrames reviews_df and other_df
# You can perform an inner join like this:
joined_df = column_listings_df.join(column_reviews_df, on='listing_id', how='inner')

# To show the result
joined_df.show()



+----------+--------------------+--------------------+-------+--------------------+---------+----------+-----------------+-------------------+-------------------------+----------------------+----------------------------+--------+---------+---------+------------+--------------+--------+----+---------+-----+--------------+--------------+----------------+-----------------+---------------------+----------------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+------------------------------+-----------------+--------+----------+-----------+-------------+--------------------+
|listing_id|         listing_url|                name|host_id|            host_url|host_name|host_since|host_is_superhost|host_listings_count|host_total_listings_count|neighbourhood_cleansed|neighbourhood_group_cleansed|latitude|longitude|room_type|accommodates|bathrooms_text|b

In [ ]:
cleaned_joined_df = joined_df.dropna(subset=['latitude'])
cleaned_joined_df.show()

+----------+--------------------+--------------------+-------+--------------------+---------+----------+-----------------+-------------------+-------------------------+----------------------+----------------------------+--------+---------+---------------+------------+--------------+--------+----+-------------+-----------------+--------------+--------------------+----------------+--------------------+---------------------+----------------------+--------------------+--------------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+------------------------------+-----------------+------------------+----------+-----------+-------------+--------------------+
|listing_id|         listing_url|                name|host_id|            host_url|host_name|host_since|host_is_superhost|host_listings_count|host_total_listings_count|neighbourhood_cleansed|neighbourhood_group_cleansed|

In [ ]:
cleaned_joined_df.count()

752323

In [ ]:
# Don't forget to stop the Spark session when you're done
spark.stop()

# New Section

# New Section